
# CNN Model Implementation and Hybrid Model in PyTorch

## Overview
This notebook covers two tasks:
1. Implementation of two well-known CNN architectures from scratch and comparison with PyTorch built-in models.
2. Creating a hybrid model combining the best features of the two architectures to achieve better performance.

### Task 1: Implementing VGG and ResNet from Scratch
- We'll choose two models: **VGG** and **ResNet**.
- Implement them from scratch and compare their performance with the built-in versions in PyTorch.

### Task 2: Creating a Hybrid Model
- Combine features from both VGG and ResNet to create a hybrid model.
- Evaluate its performance and see if it outperforms the individual models.


In [ ]:

# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

# Device configuration (GPU/CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define basic parameters
batch_size = 64
learning_rate = 0.001
num_epochs = 10

# Dataset and DataLoader (using CIFAR-10 as an example)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# 1. VGG-like network from scratch
class VGG_Scratch(nn.Module):
    def __init__(self):
        super(VGG_Scratch, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(256 * 4 * 4, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 10)  # CIFAR-10 has 10 classes
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc_layers(x)
        return x

# Initialize the VGG-like model from scratch
model_vgg_scratch = VGG_Scratch().to(device)

# 2. ResNet-like network from scratch
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

class ResNet_Scratch(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet_Scratch, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = F.avg_pool2d(x, 4)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Initialize ResNet-like model from scratch
model_resnet_scratch = ResNet_Scratch(BasicBlock, [2, 2, 2, 2]).to(device)

# Display both model architectures
print(model_vgg_scratch)
print(model_resnet_scratch)


100%|██████████| 170M/170M [00:11<00:00, 15.3MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
VGG_Scratch(
  (conv_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU()
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )

In [ ]:

# Hybrid model combining aspects of both VGG and ResNet
class HybridModel(nn.Module):
    def __init__(self):
        super(HybridModel, self).__init__()

        # VGG-inspired convolutional layers
        self.vgg_part = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # ResNet-inspired residual block
        self.res_block = BasicBlock(128, 128)

        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Linear(128 * 16 * 16, 512),
            nn.ReLU(),
            nn.Linear(512, 10)  # 10 classes for CIFAR-10
        )

    def forward(self, x):
        x = self.vgg_part(x)
        x = self.res_block(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Initialize the hybrid model
model_hybrid = HybridModel().to(device)

# Display the hybrid model architecture
print(model_hybrid)


HybridModel(
  (vgg_part): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (res_block): BasicBlock(
    (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (shortcut): Sequential()
  )
  (fc): Sequential(
    (0): Linear(in_features=32768, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
# Training function
def train(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    total_step = len(train_loader)
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')

# Testing function to calculate accuracy
def test(model, test_loader):
    model.eval()  # Evaluation mode
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        return accuracy

# Loss and optimizer for all models
criterion = nn.CrossEntropyLoss()

# 1. Train and test VGG-like model from scratch
optimizer_vgg = optim.Adam(model_vgg_scratch.parameters(), lr=learning_rate)
print("Training VGG-like model from scratch...")
train(model_vgg_scratch, train_loader, criterion, optimizer_vgg, num_epochs)
accuracy_vgg_scratch = test(model_vgg_scratch, test_loader)
print(f'VGG-like Model from scratch Test Accuracy: {accuracy_vgg_scratch:.2f}%')

# 2. Train and test ResNet-like model from scratch
optimizer_resnet = optim.Adam(model_resnet_scratch.parameters(), lr=learning_rate)
print("Training ResNet-like model from scratch...")
train(model_resnet_scratch, train_loader, criterion, optimizer_resnet, num_epochs)
accuracy_resnet_scratch = test(model_resnet_scratch, test_loader)
print(f'ResNet-like Model from scratch Test Accuracy: {accuracy_resnet_scratch:.2f}%')

# 3. Train and test Hybrid model
optimizer_hybrid = optim.Adam(model_hybrid.parameters(), lr=learning_rate)
print("Training Hybrid model...")
train(model_hybrid, train_loader, criterion, optimizer_hybrid, num_epochs)
accuracy_hybrid = test(model_hybrid, test_loader)
print(f'Hybrid Model Test Accuracy: {accuracy_hybrid:.2f}%')

# Compare with PyTorch built-in VGG and ResNet
from torchvision import models

# 4. PyTorch built-in VGG model (using VGG11)
model_vgg_builtin = models.vgg11(pretrained=False, num_classes=10).to(device)
optimizer_vgg_builtin = optim.Adam(model_vgg_builtin.parameters(), lr=learning_rate)
print("Training PyTorch built-in VGG model...")
train(model_vgg_builtin, train_loader, criterion, optimizer_vgg_builtin, num_epochs)
accuracy_vgg_builtin = test(model_vgg_builtin, test_loader)
print(f'PyTorch built-in VGG Model Test Accuracy: {accuracy_vgg_builtin:.2f}%')

# 5. PyTorch built-in ResNet model (using ResNet18)
model_resnet_builtin = models.resnet18(pretrained=False, num_classes=10).to(device)
optimizer_resnet_builtin = optim.Adam(model_resnet_builtin.parameters(), lr=learning_rate)
print("Training PyTorch built-in ResNet model...")
train(model_resnet_builtin, train_loader, criterion, optimizer_resnet_builtin, num_epochs)
accuracy_resnet_builtin = test(model_resnet_builtin, test_loader)
print(f'PyTorch built-in ResNet Model Test Accuracy: {accuracy_resnet_builtin:.2f}%')

# Final performance comparison
print(f"VGG-like Model (Scratch) Accuracy: {accuracy_vgg_scratch:.2f}%")
print(f"ResNet-like Model (Scratch) Accuracy: {accuracy_resnet_scratch:.2f}%")
print(f"Hybrid Model Accuracy: {accuracy_hybrid:.2f}%")
print(f"PyTorch Built-in VGG Model Accuracy: {accuracy_vgg_builtin:.2f}%")
print(f"PyTorch Built-in ResNet Model Accuracy: {accuracy_resnet_builtin:.2f}%")


Training VGG-like model from scratch...
Epoch [1/10], Step [100/782], Loss: 2.2074
Epoch [1/10], Step [200/782], Loss: 1.8572
Epoch [1/10], Step [300/782], Loss: 1.7206
Epoch [1/10], Step [400/782], Loss: 1.7597
Epoch [1/10], Step [500/782], Loss: 1.6427
Epoch [1/10], Step [600/782], Loss: 1.3949
Epoch [1/10], Step [700/782], Loss: 1.3690
Epoch [2/10], Step [100/782], Loss: 1.3306
Epoch [2/10], Step [200/782], Loss: 1.2617
Epoch [2/10], Step [300/782], Loss: 1.0859
Epoch [2/10], Step [400/782], Loss: 1.2066
Epoch [2/10], Step [500/782], Loss: 1.1758
Epoch [2/10], Step [600/782], Loss: 1.3204
Epoch [2/10], Step [700/782], Loss: 1.2354
Epoch [3/10], Step [100/782], Loss: 1.3279
Epoch [3/10], Step [200/782], Loss: 1.0677
Epoch [3/10], Step [300/782], Loss: 1.1709
Epoch [3/10], Step [400/782], Loss: 1.3054
Epoch [3/10], Step [500/782], Loss: 0.9375
Epoch [3/10], Step [600/782], Loss: 1.2501
Epoch [3/10], Step [700/782], Loss: 1.0430
Epoch [4/10], Step [100/782], Loss: 1.1043
Epoch [4/10], 

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Training PyTorch built-in VGG model...
Epoch [1/10], Step [100/782], Loss: 2.2824
Epoch [1/10], Step [200/782], Loss: 1.9505
Epoch [1/10], Step [300/782], Loss: 1.8361
Epoch [1/10], Step [400/782], Loss: 1.8652
Epoch [1/10], Step [500/782], Loss: 1.6995
Epoch [1/10], Step [600/782], Loss: 1.6538
Epoch [1/10], Step [700/782], Loss: 1.4796
Epoch [2/10], Step [100/782], Loss: 1.6612
Epoch [2/10], Step [200/782], Loss: 1.1077
Epoch [2/10], Step [300/782], Loss: 1.3389
Epoch [2/10], Step [400/782], Loss: 1.7847
Epoch [2/10], Step [500/782], Loss: 1.2431
Epoch [2/10], Step [600/782], Loss: 1.0621
Epoch [2/10], Step [700/782], Loss: 1.2789
Epoch [3/10], Step [100/782], Loss: 1.1831
Epoch [3/10], Step [200/782], Loss: 1.1176
Epoch [3/10], Step [300/782], Loss: 1.1273
Epoch [3/10], Step [400/782], Loss: 0.9448
Epoch [3/10], Step [500/782], Loss: 1.2227
Epoch [3/10], Step [600/782], Loss: 0.9243
Epoch [3/10], Step [700/782], Loss: 0.8321
Epoch [4/10], Step [100/782], Loss: 0.9763
Epoch [4/10], S